In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

/home/lmusher/array1/miniconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.nearest_centroid module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
data = "./W_po_17rm.snps.hdf5"

In [3]:
imap = {
#"ref": ["reference"],
"Inam": ["W_po_A815_In", "W_po_T23485_In","W_po_T3645_In","W_po_T4097_In"], #"W_po_A16828_In","W_po_T12362_In",
"Puru": ["W_po_A424_pu","W_po_A7736_pu","W_po_A101_pu","W_po_T12400_pu",  "W_po_T13148_pu", "W_po_A9697_pu","W_po_A10701_pu","W_po_A1381_pu","W_po_A265_pu", "W_po_A2725_pu", "W_po_A2749_pu", "W_po_A2759_pu"],
"JiGu": ["W_po_A308_jigu", "W_po_T7625_jigu", "W_po_T7627_jigu","W_po_T15888_jigu", "W_po_T15889_jigu","W_po_A3264_jigu", "W_po_A334_jigu", "W_po_A368_jigu"],
"Mach": ["W_po_A465_ma","W_po_T2190_ma",  "W_po_T3157_ma",  "W_po_T388_ma",  "W_po_A882_ma", "W_po_J439_ma",  "W_po_A894_ma", "W_po_A896_ma", "W_po_A898_ma", "W_po_A902_ma","W_po_A472_ma","W_po_A24413_ma","W_po_A4235_ma","W_po_J163_ma", "W_po_J192_ma", "W_po_J202_ma", "W_po_J209_ma", "W_po_J222_ma", "W_po_J226_ma", "W_po_J302_ma"], #"W_po_T6424_ma", "W_po_J448_ma", "W_po_J453_ma",
"Roar": ["W_po_J335_roar", "W_po_J340_roar", "W_po_J391_roar", "W_po_J421_roar","W_po_J644_roar", "W_po_J766_roar", "W_po_J768_roar"],
"ArSu": ["W_po_A891_arsu","W_po_80798_arsu", "W_po_81036_arsu", "W_po_85474_arsu", "W_po_86085_arsu","W_po_J561_arsu", "W_po_J563_arsu", "W_po_J610_arsu"],
"SuTa": ["W_po_A16298_suta","W_po_T11887_suta","W_po_81265_suta", "W_po_T10153_suta", "W_po_T10234_suta"], #"W_po_T14615_suta","W_po_85827_suta","W_po_A12265_suta",
"Para": [ "W_po_T12793_pa", "W_po_T7132_pa",  "W_po_T9346_pa","W_po_T10694_pa","W_po_25484_pa","W_po_A7346_pa","W_po_A12284_pa",  "W_po_A14569_pa", "W_po_A16227_pa"] #"W_po_A9868_pa",
}
                      
# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [4]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [4]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="W_po_str",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

110 previous results loaded for run [W_po_str]
Samples: 73
Sites before filtering: 2197855
Filtered (indels): 0
Filtered (bi-allel): 69363
Filtered (mincov): 2047527
Filtered (minmap): 1537795
Filtered (combined): 2053756
Sites after filtering: 144099
Sites containing missing values: 126160 (87.55%)
Missing values in SNP matrix: 258162 (2.45%)


In [5]:
struct.mainparams.burnin = 50000
struct.mainparams.numreps = 250000
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/W_po_17rm_outfiles/analysis-structure/tmp-W_po_str-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/W_po_17rm_outfiles/analysis-structure/tmp-W_po_str-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/W_po_17rm_outfiles/analysis-structure/tmp-W_po_str-<built-in function abs>-1.strfile.txt')

In [7]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####################] 100% 3 days, 16:42:43 | running 110 structure jobs 


In [8]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
etable

,Nreps,lnPK,lnPPK,deltaK,estLnProbMean,estLnProbStdev
2,10,0.00,0.00,0.000000,-134955.40,8.717820e+02
3,10,164.21,92980.51,78.483099,-134791.19,1.184720e+03
4,10,-92816.30,642.46,0.003865,-227607.49,1.662417e+05
5,10,-92173.84,621707.43,1.989234,-319781.33,3.125360e+05
6,10,-713881.27,201476.86,0.365315,-1033662.60,5.515156e+05
7,10,-915358.13,1174715.43,1.032550,-1949020.73,1.137684e+06
8,10,259357.30,253450.35,0.128489,-1689663.43,1.972550e+06
9,10,512807.65,2331143.00,2.282645,-1176855.78,1.021246e+06
10,10,-1818335.35,130251.13,0.052551,-2995191.13,2.478570e+06
11,10,-1948586.48,4791577.19,2.304443,-4943777.61,2.079278e+06


In [9]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tabe69ccff83548dab2683ad58b8f0b4b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 9 10 11 12 K (N ancestral populations) 0 1000000 2000000 3000000 4000000 5000000 estLnProbMean 0 50 100 deltaK

In [39]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [40]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [41]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="td31501e79df54eaf9ac0f061a7bb3f03" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A265_pu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_A16298_suta W_po_T11887_suta W_po_81265_suta W_po_T10153_suta W_po_T10234_suta W_po_T12793_pa W_po_T7132_pa W_po_T9346_pa W_po_T10694_pa W_po_25484_pa W_po_A7346_pa W_po_A12284_pa W_po_A14569_pa W_po_A16227_pa 0.0 0.5 1.0

In [45]:
k = 3
table = struct.get_clumpp_table(k)

# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K3] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t66e1fbdbbd854130bf0fcc4e6d0f8b24" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A265_pu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_A16298_suta W_po_T11887_suta W_po_81265_suta W_po_T10153_suta W_po_T10234_suta W_po_T12793_pa W_po_T7132_pa W_po_T9346_pa W_po_T10694_pa W_po_25484_pa W_po_A7346_pa W_po_A12284_pa W_po_A14569_pa W_po_A16227_pa 0.0 0.5 1.0

In [46]:
k = 4
table = struct.get_clumpp_table(k)

# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K4] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t53eb9a0973034621a9d88677ccbc7e1f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A265_pu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_A16298_suta W_po_T11887_suta W_po_81265_suta W_po_T10153_suta W_po_T10234_suta W_po_T12793_pa W_po_T7132_pa W_po_T9346_pa W_po_T10694_pa W_po_25484_pa W_po_A7346_pa W_po_A12284_pa W_po_A14569_pa W_po_A16227_pa 0.0 0.5 1.0

In [47]:
k = 5
table = struct.get_clumpp_table(k)

# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K5] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t06e4cca0ae50478bb776aa0b740565d9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A265_pu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_A16298_suta W_po_T11887_suta W_po_81265_suta W_po_T10153_suta W_po_T10234_suta W_po_T12793_pa W_po_T7132_pa W_po_T9346_pa W_po_T10694_pa W_po_25484_pa W_po_A7346_pa W_po_A12284_pa W_po_A14569_pa W_po_A16227_pa 0.0 0.5 1.0

In [48]:
k = 6
table = struct.get_clumpp_table(k)

# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K6] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="ta0c684385c044cf18d1a51a69dccfef6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A265_pu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_A16298_suta W_po_T11887_suta W_po_81265_suta W_po_T10153_suta W_po_T10234_suta W_po_T12793_pa W_po_T7132_pa W_po_T9346_pa W_po_T10694_pa W_po_25484_pa W_po_A7346_pa W_po_A12284_pa W_po_A14569_pa W_po_A16227_pa 0.0 0.5 1.0

In [16]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [51]:
imap = {
#"ref": ["reference"],
"Inam": ["W_po_A815_In", "W_po_T23485_In","W_po_T3645_In","W_po_T4097_In"], #"W_po_A16828_In","W_po_T12362_In",
"Puru": ["W_po_A424_pu","W_po_A7736_pu","W_po_A101_pu","W_po_T12400_pu",  "W_po_T13148_pu", "W_po_A9697_pu","W_po_A10701_pu","W_po_A1381_pu","W_po_A2725_pu", "W_po_A2749_pu", "W_po_A2759_pu"],
"JiGu": ["W_po_A308_jigu", "W_po_T7625_jigu", "W_po_T7627_jigu","W_po_T15888_jigu", "W_po_T15889_jigu","W_po_A3264_jigu", "W_po_A334_jigu", "W_po_A368_jigu","W_po_A265_pu", ],
"Mach": ["W_po_A465_ma","W_po_T2190_ma",  "W_po_T3157_ma",  "W_po_T388_ma",  "W_po_A882_ma", "W_po_J439_ma",  "W_po_A894_ma", "W_po_A896_ma", "W_po_A898_ma", "W_po_A902_ma","W_po_A472_ma","W_po_A24413_ma","W_po_A4235_ma","W_po_J163_ma", "W_po_J192_ma", "W_po_J202_ma", "W_po_J209_ma", "W_po_J222_ma", "W_po_J226_ma", "W_po_J302_ma"], #"W_po_T6424_ma", "W_po_J448_ma", "W_po_J453_ma",
"Roar": ["W_po_J335_roar", "W_po_J340_roar", "W_po_J391_roar", "W_po_J421_roar","W_po_J644_roar", "W_po_J766_roar", "W_po_J768_roar"],
"ArSu": ["W_po_A891_arsu","W_po_80798_arsu", "W_po_81036_arsu", "W_po_85474_arsu", "W_po_86085_arsu","W_po_J561_arsu", "W_po_J563_arsu", "W_po_J610_arsu"],
"SuTa": ["W_po_A16298_suta","W_po_T11887_suta","W_po_81265_suta", "W_po_T10153_suta", "W_po_T10234_suta"], #"W_po_T14615_suta","W_po_85827_suta","W_po_A12265_suta",
"Para": [ "W_po_T12793_pa", "W_po_T7132_pa",  "W_po_T9346_pa","W_po_T10694_pa","W_po_25484_pa","W_po_A7346_pa","W_po_A12284_pa",  "W_po_A14569_pa", "W_po_A16227_pa"] #"W_po_A9868_pa",
}
                      
# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [52]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 73
Sites before filtering: 2197855
Filtered (indels): 0
Filtered (bi-allel): 69363
Filtered (mincov): 1175528
Filtered (minmap): 1547510
Filtered (combined): 1574300
Sites after filtering: 623555
Sites containing missing values: 605616 (97.12%)
Missing values in SNP matrix: 5602376 (12.31%)
Imputation: 'sampled'; (0, 1, 2) = 73.4%, 4.2%, 22.3%


In [53]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 41788/623555


In [54]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b9c1befae10>,
 <toyplot.mark.Point at 0x2b9c3c02c908>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t1b526c165d344181886e49ad3bac5161" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_25484_pa W_po_80798_arsu W_po_81036_arsu W_po_81265_suta W_po_85474_arsu W_po_86085_arsu W_po_A101_pu W_po_A10701_pu W_po_A12284_pa W_po_A1381_pu W_po_A14569_pa W_po_A16227_pa W_po_A16298_suta W_po_A24413_ma W_po_A265_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A4235_ma W_po_A424_pu W_po_A465_ma W_po_A472_ma W_po_A7346_pa W_po_A7736_pu W_po_A815_In W_po_A882_ma W_po_A891_arsu W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A9697_pu W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J439_ma W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_T10153_suta W_po_T10234_suta W_po_T10694_pa W_po_T11887_suta W_po_T12400_pu W_po_T12793_pa W_po_T13148_pu W_po_T15888_jigu W_po_T15889_jigu W_po_T2190_ma W_po_T23485_In W_po_T3157_ma W_po_T3645_In W_po_T388_ma W_po_T4097_In W_po_T7132_pa W_po_T7625_jigu W_po_T7627_jigu W_po_T9346_pa -20 0 20 40 60 PC0 (27.9%) explained -10 0 10 20 30 PC1 (6.5%) explained Inam Puru JiGu Mach Roar ArSu SuTa Para

In [55]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("W_po_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
W_po_25484_pa,62.56,-7.67,-16.68,-0.19,0.17,-0.05,-0.43,-0.45,0.08,1.33
W_po_80798_arsu,-15.19,-4.88,-0.03,-14.83,-0.67,18.01,-8.15,3.47,1.29,1.73
W_po_81036_arsu,-14.75,-4.63,-0.17,-14.08,-0.80,14.00,-5.67,2.95,0.74,-1.91
W_po_81265_suta,50.49,1.81,38.13,-0.04,-1.46,0.22,0.15,0.85,-0.05,0.66
W_po_85474_arsu,-14.88,-5.10,-0.26,-15.23,-0.65,14.67,-6.87,1.97,5.86,-0.29
W_po_86085_arsu,-15.00,-4.93,0.52,-13.26,-1.49,16.22,-5.16,1.91,2.69,2.96
W_po_A101_pu,-5.39,28.78,-3.67,1.60,-10.76,-2.01,-0.30,-2.68,6.38,6.41
W_po_A10701_pu,-5.91,26.78,-3.63,-0.50,-8.28,-1.76,1.49,-1.40,0.97,-0.42
W_po_A12284_pa,61.97,-7.27,-15.43,0.04,-0.75,-0.25,0.01,0.08,0.03,-0.30
W_po_A1381_pu,-6.33,26.21,-3.74,0.36,-9.20,0.25,0.91,1.07,6.67,-9.38


In [56]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 1);

Subsampling SNPs: 41788/623555


<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc60e6ede13be4c20a2e33ae0d9c7aa74" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <circle r

In [23]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 1);

Kmeans clustering: iter=0, K=7, mincov=0.9, minmap={'global': 0.5}
Samples: 73
Sites before filtering: 2197855
Filtered (indels): 0
Filtered (bi-allel): 69363
Filtered (mincov): 1874495
Filtered (minmap): 1175528
Filtered (combined): 1887914
Sites after filtering: 309941
Sites containing missing values: 292002 (94.21%)
Missing values in SNP matrix: 1162436 (5.14%)
Imputation: 'sampled'; (0, 1, 2) = 75.0%, 5.3%, 19.8%
{0: ['W_po_A265_pu', 'W_po_A308_jigu', 'W_po_A3264_jigu', 'W_po_A334_jigu', 'W_po_A368_jigu', 'W_po_A882_ma', 'W_po_T15888_jigu', 'W_po_T15889_jigu', 'W_po_T388_ma', 'W_po_T7625_jigu', 'W_po_T7627_jigu'], 1: ['W_po_85474_arsu', 'W_po_86085_arsu', 'W_po_A24413_ma', 'W_po_A896_ma', 'W_po_J335_roar', 'W_po_J340_roar', 'W_po_J391_roar', 'W_po_J421_roar', 'W_po_J561_arsu', 'W_po_J610_arsu', 'W_po_J644_roar', 'W_po_J766_roar', 'W_po_J768_roar'], 2: ['W_po_81265_suta', 'W_po_A16298_suta', 'W_po_T10153_suta', 'W_po_T10234_suta', 'W_po_T11887_suta'], 3: ['W_po_A101_pu', 'W_po_A1070

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t3decd08bb0b244b2ad840356eae2d974" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> </

In [24]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "W_po_PCA-kmeans-7.pdf")

In [57]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [64]:
pca.run_tsne(subsample=True, perplexity=9.0, n_iter=1000000, seed=224)

Subsampling SNPs: 41788/623555


In [65]:
pca.draw(size=8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t40742b933f1a4c4ab360ab05b6dfdcd9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_25484_pa W_po_80798_arsu W_po_81036_arsu W_po_81265_suta W_po_85474_arsu W_po_86085_arsu W_po_A101_pu W_po_A10701_pu W_po_A12284_pa W_po_A1381_pu W_po_A14569_pa W_po_A16227_pa W_po_A16298_suta W_po_A24413_ma W_po_A265_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A4235_ma W_po_A424_pu W_po_A465_ma W_po_A472_ma W_po_A7346_pa W_po_A7736_pu W_po_A815_In W_po_A882_ma W_po_A891_arsu W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A9697_pu W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J439_ma W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_T10153_suta W_po_T10234_suta W_po_T10694_pa W_po_T11887_suta W_po_T12400_pu W_po_T12793_pa W_po_T13148_pu W_po_T15888_jigu W_po_T15889_jigu W_po_T2190_ma W_po_T23485_In W_po_T3157_ma W_po_T3645_In W_po_T388_ma W_po_T4097_In W_po_T7132_pa W_po_T7625_jigu W_po_T7627_jigu W_po_T9346_pa -200 0 200 400 TNSE component 1 -200 0 200 400 TNSE component 2 Inam Puru JiGu Mach Roar ArSu SuTa Para

In [28]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "W_po_TSNE_perp9.pdf")

In [29]:
import ipyrad.analysis as ipa
import toyplot

In [30]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 73
Sites before filtering: 2197855
Filtered (indels): 0
Filtered (bi-allel): 69363
Filtered (mincov): 1175528
Filtered (minmap): 1547510
Filtered (combined): 1574300
Sites after filtering: 623555
Sites containing missing values: 605616 (97.12%)
Missing values in SNP matrix: 5602376 (12.31%)
Imputation: 'sampled'; (0, 1, 2) = 73.5%, 4.2%, 22.3%


In [31]:
# save to a CSV file
dist.dists.to_csv("W_po_distances.csv")

In [32]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "W_po_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [33]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [34]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="t11721cd91a6848329b94091240b76468" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.120262 0.102952 0.106528 0.116954 0.114651 0.113640 0.115146 0.115050 0.113466 0.113964 0.115432 0.117047 0.114798 0.114463 0.136705 0.136652 0.136998 0.138273 0.136246 0.136025 0.136949 0.136469 0.136971 0.131102 0.131947 0.131979 0.133231 0.135281 0.133922 0.133069 0.131129 0.132147 0.132187 0.132850 0.132784 0.131693 0.131899 0.132094 0.130088 0.132134 0.132551 0.132750 0.133841 0.132452 0.132497 0.157677 0.132622 0.132577 0.133518 0.132766 0.133337 0.134545 0.133639 0.133518 0.132601 0.133428 0.133029 0.132245 0.216408 0.216408 0.216085 0.217184 0.216010 0.227417 0.227361 0.227635 0.226994 0.227887 0.227699 0.227544 0.227621 0.227167 W_po_A815_In 1 0.120262 0.000000 0.118347 0.118485 0.146860 0.145808 0.144521 0.145796 0.146011 0.144074 0.145076 0.145809 0.147030 0.145593 0.145235 0.164083 0.164149 0.164457 0.165081 0.163721 0.163227 0.164369 0.163549 0.164504 0.158409 0.159049 0.159190 0.160052 0.163310 0.161022 0.160752 0.158462 0.158968 0.159162 0.159997 0.160310 0.159065 0.159688 0.159370 0.157175 0.158862 0.158868 0.159834 0.161036 0.159670 0.160218 0.184689 0.159975 0.159673 0.160643 0.159660 0.161004 0.161840 0.160812 0.160794 0.159716 0.160659 0.160392 0.159466 0.213094 0.212860 0.212666 0.213999 0.212498 0.223414 0.223393 0.223475 0.222696 0.223829 0.223866 0.223226 0.223656 0.222843 W_po_T23485_In 2 0.102952 0.118347 0.000000 0.100517 0.115898 0.114333 0.112636 0.115191 0.113825 0.113315 0.114128 0.114399 0.116299 0.113615 0.113873 0.137042 0.137271 0.138034 0.138758 0.136875 0.136484 0.137391 0.137079 0.137641 0.131551 0.132553 0.133387 0.133683 0.135868 0.134548 0.133698 0.131873 0.132843 0.132720 0.133568 0.133364 0.132479 0.132399 0.132874 0.130781 0.132615 0.132811 0.133254 0.134460 0.133645 0.133773 0.158812 0.133513 0.133860 0.134128 0.133892 0.134591 0.135626 0.134582 0.134407 0.133412 0.134590 0.133796 0.133214 0.217062 0.217071 0.216845 0.217565 0.216558 0.228225 0.228663 0.228318 0.227507 0.228647 0.228812 0.228111 0.228400 0.227545 W_po_T3645_In 3 0.106528 0.118485 0.100517 0.000000 0.115760 0.113897 0.112601 0.114572 0.114162 0.112234 0.113682 0.114441 0.115398 0.113005 0.113533 0.136259 0.136742 0.136604 0.137266 0.136051 0.135499 0.136506 0.136164 0.136862 0.130187 0.131193 0.130949 0.132664 0.135053 0.133271 0.132290 0.130584 0.131307 0.130555 0.131798 0.131927 0.130664 0.130968 0.131013 0.129238 0.130768 0.131445 0.131656 0.133026 0.131881 0.131563 0.156490 0.131971 0.131990 0.132816 0.131900 0.132850 0.133570 0.132817 0.133015 0.131850 0.132761 0.132121 0.131440 0.215513 0.215458 0.215476 0.216154 0.215352 0.226436 0.226662 0.226830 0.225916 0.226979 0.227257 0.226444 0.226774 0.226201 W_po_T4097_In 4 0.116954 0.146860 0.115898 0.115760 0.000000 0.107779 0.103061 0.107229 0.110734 0.106751 0.105659 0.106343 0.108837 0.107410 0.106911 0.131613 0.131826 0.132061 0.132922 0.131296 0.131321 0.131966 0.131583 0.131966 0.125484 0.126643 0.126964 0.128143 0.130208 0.128487 0.128638 0.126281 0.126978 0.127285 0.127681 0.127673 0.126480 0.127288 0.127294 0.124980 0.126340 0.127764 0.127007 0.128425 0.128184 0.127761 0.152732 0.127681 0.127536 0.128754 0.128043 0.128720 0.129030 0.129034 0.129299 0.127711 0.128975 0.127835 0.127807 0.215925 0.216043 0.216590 0.217176 0.216268 0.227627 0.227760 0.227197 0.226691 0.227943 0.227900 0.226747 0.227465 0.227149 W_po_A424_pu 5 0.114651 0.145808 0.114333 0.113897 0.107779 0.000000 0.101688 0.105484 0.107392 0.104464 0.103